
<h2 id="Analyze-A/B-Test-Results">Analyze A/B Test Results<a class="anchor-link" href="#Analyze-A/B-Test-Results">¶</a></h2><p>You may either submit your notebook through the workspace here, or you may work from your local machine and submit through the next page.  Either way assure that your code passes the project <a href="https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric">RUBRIC</a>.  <strong>Please save regularly.</strong></p>
<p>This project will assure you have mastered the subjects covered in the statistics lessons.  The hope is to have this project be as comprehensive of these topics as possible.  Good luck!</p>
<h2 id="Table-of-Contents">Table of Contents<a class="anchor-link" href="#Table-of-Contents">¶</a></h2><ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#probability">Part I - Probability</a></li>
<li><a href="#ab_test">Part II - A/B Test</a></li>
<li><a href="#regression">Part III - Regression</a></li>
</ul>
<p><a id="intro"></a></p>
<h3 id="Introduction">Introduction<a class="anchor-link" href="#Introduction">¶</a></h3><p>A/B tests are very commonly performed by data analysts and data scientists.  It is important that you get some practice working with the difficulties of these</p>
<p>For this project, you will be working to understand the results of an A/B test run by an e-commerce website.  Your goal is to work through this notebook to help the company understand if they should implement the new page, keep the old page, or perhaps run the experiment longer to make their decision.</p>
<p><strong>As you work through this notebook, follow along in the classroom and answer the corresponding quiz questions associated with each question.</strong> The labels for each classroom concept are provided for each question.  This will assure you are on the right track as you work through the project, and you can feel more confident in your final submission meeting the criteria.  As a final check, assure you meet all the criteria on the <a href="https://review.udacity.com/#!/projects/37e27304-ad47-4eb0-a1ab-8c12f60e43d0/rubric">RUBRIC</a>.</p>
<p><a id="probability"></a></p>
<h4 id="Part-I---Probability">Part I - Probability<a class="anchor-link" href="#Part-I---Probability">¶</a></h4><p>To get started, let's import our libraries.</p>


In [ ]:

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
#We are setting the seed to assure you get the same answers on quizzes as we set up
random.seed(42)




<p><code>1.</code> Now, read in the <code>ab_data.csv</code> data. Store it in <code>df</code>.  <strong>Use your dataframe to answer the questions in Quiz 1 of the classroom.</strong></p>
<p>a. Read in the dataset and take a look at the top few rows here:</p>


In [ ]:

df = pd.read_csv('ab_data.csv')
df.head()




<p>b. Use the cell below to find the number of rows in the dataset.</p>


In [ ]:

total = df.shape[0]
total




<p>c. The number of unique users in the dataset.</p>


In [ ]:

unique_users = df.user_id.nunique()
unique_users




<p>d. The proportion of users converted.</p>


In [ ]:

did_converted = df.query('converted == 1').shape[0]
did_converted/unique_users




<p>e. The number of times the <code>new_page</code> and <code>treatment</code> don't match.</p>


In [ ]:

df.query('landing_page == "new_page" & group != "treatment"').shape[0] + df.query('landing_page == "old_page" & group != "control"').shape[0]




<p>f. Do any of the rows have missing values?</p>


In [ ]:

df.info()




<p><code>2.</code> For the rows where <strong>treatment</strong> does not match with <strong>new_page</strong> or <strong>control</strong> does not match with <strong>old_page</strong>, we cannot be sure if this row truly received the new or old page.  Use <strong>Quiz 2</strong> in the classroom to figure out how we should handle these rows.</p>
<p>a. Now use the answer to the quiz to create a new dataset that meets the specifications from the quiz.  Store your new dataframe in <strong>df2</strong>.</p>


In [ ]:

df1 = df.drop(df.query('landing_page == "new_page" & group != "treatment"').index)
df2 = df1.drop(df.query('landing_page == "old_page" & group != "control"').index)



In [ ]:

# Double Check all of the correct rows were removed - this should be 0
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]




<p><code>3.</code> Use <strong>df2</strong> and the cells below to answer questions for <strong>Quiz3</strong> in the classroom.</p>



<p>a. How many unique <strong>user_id</strong>s are in <strong>df2</strong>?</p>


In [ ]:

df2.nunique()




<p>b. There is one <strong>user_id</strong> repeated in <strong>df2</strong>.  What is it?</p>


In [ ]:

df2[df2.user_id.duplicated()]




<p>c. What is the row information for the repeat <strong>user_id</strong>?</p>



<p>d. Remove <strong>one</strong> of the rows with a duplicate <strong>user_id</strong>, but keep your dataframe as <strong>df2</strong>.</p>


In [ ]:

df2 = df2.drop(df2[df2.user_id.duplicated()].index)




<p><code>4.</code> Use <strong>df2</strong> in the cells below to answer the quiz questions related to <strong>Quiz 4</strong> in the classroom.</p>
<p>a. What is the probability of an individual converting regardless of the page they receive?</p>


In [ ]:

df2[df2['converted']==1].shape[0]/df2.shape[0]




<p>b. Given that an individual was in the <code>control</code> group, what is the probability they converted?</p>


In [ ]:

(df2.query('group == "control"')['converted'] == 1).mean()




<p>c. Given that an individual was in the <code>treatment</code> group, what is the probability they converted?</p>


In [ ]:

(df2.query('group == "treatment"')['converted'] == 1).mean()




<p>d. What is the probability that an individual received the new page?</p>


In [ ]:

df2.query('landing_page == "new_page"').shape[0]/df2.shape[0]




<p>e. Consider your results from parts (a) through (d) above, and explain below whether you think there is sufficient evidence to conclude that the new treatment page leads to more conversions.</p>



<p><strong>no, we cannot say that new treatment page leads  to more conversion as there is a small diffrence between conversion rates of both</strong></p>



<p><a id="ab_test"></a></p>
<h3 id="Part-II---A/B-Test">Part II - A/B Test<a class="anchor-link" href="#Part-II---A/B-Test">¶</a></h3><p>Notice that because of the time stamp associated with each event, you could technically run a hypothesis test continuously as each observation was observed.</p>
<p>However, then the hard question is do you stop as soon as one page is considered significantly better than another or does it need to happen consistently for a certain amount of time?  How long do you run to render a decision that neither page is better than another?</p>
<p>These questions are the difficult parts associated with A/B tests in general.</p>
<p><code>1.</code> For now, consider you need to make the decision just based on all the data provided.  If you want to assume that the old page is better unless the new page proves to be definitely better at a Type I error rate of 5%, what should your null and alternative hypotheses be?  You can state your hypothesis in terms of words or in terms of <strong>$p_{old}$</strong> and <strong>$p_{new}$</strong>, which are the converted rates for the old and new pages.</p>



<p>Null hypothesis: conversion rate of the old page is more than or equal the new page 
alternative hypothesis : conversion rate of the new page is more than the old page</p>
$$ H_{0}: p_{old} - p_{new} = 0$$$$ H_{1}: p_{old} - p_{new} \neq 0$$<p></p>



<p><code>2.</code> Assume under the null hypothesis, $p_{new}$ and $p_{old}$ both have "true" success rates equal to the <strong>converted</strong> success rate regardless of page - that is $p_{new}$ and $p_{old}$ are equal. Furthermore, assume they are equal to the <strong>converted</strong> rate in <strong>ab_data.csv</strong> regardless of the page. <br/><br/></p>
<p>Use a sample size for each page equal to the ones in <strong>ab_data.csv</strong>.  <br/><br/></p>
<p>Perform the sampling distribution for the difference in <strong>converted</strong> between the two pages over 10,000 iterations of calculating an estimate from the null.  <br/><br/></p>
<p>Use the cells below to provide the necessary parts of this simulation.  If this doesn't make complete sense right now, don't worry - you are going to work through the problems below to complete this problem.  You can use <strong>Quiz 5</strong> in the classroom to make sure you are on the right track.<br/><br/></p>



<p>a. What is the <strong>conversion rate</strong> for $p_{new}$ under the null?</p>


In [ ]:

p_new = df2.query("converted == 1").shape[0]/df2.shape[0]
p_new




<p>b. What is the <strong>conversion rate</strong> for $p_{old}$ under the null? <br/><br/></p>


In [ ]:

p_old = df2.query("converted == 1").shape[0]/df2.shape[0]
p_old




<p>c. What is $n_{new}$, the number of individuals in the treatment group?</p>


In [ ]:

n_new = df2.query('group == "treatment"').shape[0]
n_new




<p>d. What is $n_{old}$, the number of individuals in the control group?</p>


In [ ]:

n_old = df2.query('group == "control"').shape[0]
n_old




<p>e. Simulate $n_{new}$ transactions with a conversion rate of $p_{new}$ under the null.  Store these $n_{new}$ 1's and 0's in <strong>new_page_converted</strong>.</p>


In [ ]:

new_page_converted = np.random.binomial(1, p_new, n_new)
new_page_converted




<p>f. Simulate $n_{old}$ transactions with a conversion rate of $p_{old}$ under the null.  Store these $n_{old}$ 1's and 0's in 
<strong>old_page_converted</strong>.</p>


In [ ]:

old_page_converted = np.random.binomial(1, p_old, n_old)




<p>g. Find $p_{new}$ - $p_{old}$ for your simulated values from part (e) and (f).</p>


In [ ]:

new_page_converted.mean() - old_page_converted.mean()




<p>h. Create 10,000 $p_{new}$ - $p_{old}$ values using the same simulation process you used in parts (a) through (g) above. Store all 10,000 values in a NumPy array called <strong>p_diffs</strong>.</p>


In [ ]:

diffs = []
for _ in range(10000):
    new_page_converted = np.random.binomial(1, p_new, n_new)
    old_page_converted = np.random.binomial(1, p_old, n_old)
    minus = new_page_converted.mean() - old_page_converted.mean()
    diffs.append(minus)




<p>i. Plot a histogram of the <strong>p_diffs</strong>.  Does this plot look like what you expected?  Use the matching problem in the classroom to assure you fully understand what was computed here.</p>


In [ ]:

p_diffs = np.array(diffs)
plt.hist(p_diffs)




<p>j. What proportion of the <strong>p_diffs</strong> are greater than the actual difference observed in <strong>ab_data.csv</strong>?</p>


In [ ]:

actual_obs = (df2.query('group == "treatment"')['converted'] == 1).mean() - (df2.query('group == "control"')['converted'] == 1).mean()



In [ ]:

(p_diffs > actual_obs).mean()




<p><strong>We have calculated the "P-Value" in the previous steps, P-Value is the proportion of extreme values of the sample we have simulated. the greater "P-Value" resulted  the more we tend to choose the null hypothesis(shouldn't be more than 0.005 to reject the null hypothesis) and due to the very big P-Value resulted from the previous simulation so, we fail to reject the null hypothesis.
in other words, our A/B Testing result came in favour of the old page and against the new page.</strong></p>



<p>l. We could also use a built-in to achieve similar results.  Though using the built-in might be easier to code, the above portions are a walkthrough of the ideas that are critical to correctly thinking about statistical significance. Fill in the below to calculate the number of conversions for each page, as well as the number of individuals who received each page. Let <code>n_old</code> and <code>n_new</code> refer the the number of rows associated with the old page and new pages, respectively.</p>


In [ ]:

import statsmodels.api as sm

convert_old = df2[np.logical_and(df2['group']=='control', df2['converted'] == 1)].shape[0]
convert_new = df2[np.logical_and(df2['group']=='treatment', df2['converted'] == 1)].shape[0]
n_old = df2.query('group == "control"').shape[0]
n_new =df2.query('group == "treatment"').shape[0]
 



In [ ]:

convert_old, convert_new, n_old, n_new




<p>m. Now use <code>stats.proportions_ztest</code> to compute your test statistic and p-value.  <a href="https://docs.w3cub.com/statsmodels/generated/statsmodels.stats.proportion.proportions_ztest/">Here</a> is a helpful link on using the built in.</p>


In [ ]:

z_score, p_value = sm.stats.proportions_ztest([convert_old, convert_new], [n_old, n_new], alternative='smaller')
z_score, p_value




<p>n. What do the z-score and p-value you computed in the previous question mean for the conversion rates of the old and new pages?  Do they agree with the findings in parts <strong>j.</strong> and <strong>k.</strong>?</p>



<p><strong>z-score represents how many standard deviations away our data point is from the mean, our z-score is positive which suggests that our data point is on the right side of the mean line on the bell curve.
p-value of 0.9050 is very close to the p-value we computed earlier in part, and again we fail to reject null hypothesis.</strong></p>



<p><a id="regression"></a></p>
<h3 id="Part-III---A-regression-approach">Part III - A regression approach<a class="anchor-link" href="#Part-III---A-regression-approach">¶</a></h3><p><code>1.</code> In this final part, you will see that the result you achieved in the A/B test in Part II above can also be achieved by performing regression.<br/><br/></p>
<p>a. Since each row is either a conversion or no conversion, what type of regression should you be performing in this case?</p>



<p><strong>Logistic Regression</strong></p>



<p>b. The goal is to use <strong>statsmodels</strong> to fit the regression model you specified in part <strong>a.</strong> to see if there is a significant difference in conversion based on which page a customer receives. However, you first need to create in df2 a column for the intercept, and create a dummy variable column for which page each user received.  Add an <strong>intercept</strong> column, as well as an <strong>ab_page</strong> column, which is 1 when an individual receives the <strong>treatment</strong> and 0 if <strong>control</strong>.</p>


In [ ]:

df2[['new_page', 'old_page']] = pd.get_dummies(df2['landing_page'])
df2['intercept'] = 1 
df2.head()




<p>c. Use <strong>statsmodels</strong> to instantiate your regression model on the two columns you created in part b., then fit the model using the two columns you created in part <strong>b.</strong> to predict whether or not an individual converts.</p>


In [ ]:

log_mod = sm.Logit(df2['converted'], df2[['intercept', 'new_page']])
result = log_mod.fit()
result.summary2()




<p>d. Provide the summary of your model below, and use it as necessary to answer the following questions.</p>


In [ ]:

new = 1/np.exp(-0.0150)
print('new page:{}'.format(new))




<p>e. What is the p-value associated with <strong>ab_page</strong>? Why does it differ from the value you found in <strong>Part II</strong>?<br/><br/> <strong>Hint</strong>: What are the null and alternative hypotheses associated with your regression model, and how do they compare to the null and alternative hypotheses in <strong>Part II</strong>?</p>



<p><strong>p value of ab_page is approximately 0.19 in logistic regression model, while it was 0.9 in the hypothesis testing and that's because hypothesis testing is two tailed while logistic regression computing one tailed 
" 2 * (1 - p_value_one_tailed)"</strong></p>



$$ H_{0}: p_{old} - p_{new} = 0$$$$ H_{1}: p_{old} - p_{new} \neq 0$$<p></p>



<p>f. Now, you are considering other things that might influence whether or not an individual converts.  Discuss why it is a good idea to consider other factors to add into your regression model.  Are there any disadvantages to adding additional terms into your regression model?</p>



<p><strong>I think we shouldn't only look at the final results of each page and ignore other factors like the nature of marketing campaign for the institution before and after lanching the new page</strong></p>



<p>g. Now along with testing if the conversion rate changes for different pages, also add an effect based on which country a user lives in. You will need to read in the <strong>countries.csv</strong> dataset and merge together your datasets on the appropriate rows.  <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.join.html">Here</a> are the docs for joining tables.</p>
<p>Does it appear that country had an impact on conversion?  Don't forget to create dummy variables for these country columns - <strong>Hint: You will need two columns for the three dummy variables.</strong> Provide the statistical output as well as a written response to answer this question.</p>


In [ ]:

# read file and join the dfs
countries_df = pd.read_csv('./countries.csv')
df_new = countries_df.set_index('user_id').join(df2.set_index('user_id'), how='inner')
df_new.head()



In [ ]:

df_new.country.value_counts()



In [ ]:

#getting country coloumn dummies
df_new[['CA', 'UK', 'US']] = pd.get_dummies(df_new['country'])



In [ ]:

df_new.sample(6)



In [ ]:

#fitting logistic regression model
log_mod = sm.Logit(df_new['converted'], df_new[['intercept', 'US', 'UK']])
result = log_mod.fit()
result.summary2()



In [ ]:

1/np.exp(0.408), 1/np.exp(0.0507)




<p>users from UK are 0.95 times more likely to convert as compared to users from CA, while users from Us are 0.66 times more likely to more convert as compared to users from CA</p>



<p>h. Though you have now looked at the individual factors of country and page on conversion, we would now like to look at an interaction between page and country to see if there significant effects on conversion.  Create the necessary additional columns, and fit the new model.</p>
<p>Provide the summary results, and your conclusions based on the results.</p>


In [ ]:

#create the interaction columns
df_new['ab_uk'], df_new['ab_us'] = df_new['new_page']*df_new['UK'], df_new['new_page']*df_new['US']



In [ ]:

#fitting logistic regression model
log_mod = sm.Logit(df_new['converted'], df_new[['intercept', 'ab_uk', 'ab_us', 'US', 'UK']])
result = log_mod.fit()
result.summary2()



In [ ]:

uk = 1/np.exp(0.0453)
us = 1/np.exp(0.0511)
ab_uk = 1/np.exp(0.0108)
ab_us = 1/np.exp(-0.0206)
print('UK:{}, \n US:{}, ab_uk:{}, ab_us:{}'.format(uk, us, ab_uk, ab_us)) 




<p><strong>we can say that: users from UK are 0.95 times more likely to convert as compared to users from CA, while users from Us are 0.96 times more likely to more convert as compared to users from CA, and the interaction between ab_page with uk and ab_page with us is 0.99 and 1.02 respictevly </strong></p>



<h2 id="Conclusions">Conclusions<a class="anchor-link" href="#Conclusions">¶</a></h2><p>**in this project we have used <code>A/B Testing, Z-testing</code> processes, <code>Logistic Regression Model</code> in order to make our anlysis in the best quality.   We have three main stages of our analysis.</p>
<ul>
<li><strong>First, we cleaned some quality issues we dropped the duplicated and the missing values, then We perform the hypothesis testing on the dataset to get the closest answer to the truth, we calculated the p-value(which i have explained before), the result of this test came against the new page.</strong> </li>
<li><strong>We perform Z=testing in the third stage which again come against the new page.</strong></li>
<li><strong>And the final stage was the logistic regression model here we the resulted was that the new page leads to only 1.015 than the old page, so here we can confidently say that there is no reason to lanch the new page because there is no obvius diffrence between conversion of the new and the old one and better design a new third page to increase the conversion rate, also we added the country column in another step to see it's impact nad we found out that users from UK are 0.95 times more likely to convert as compared to users from CA, while users from Us are 0.66 times more likely to more convert as compared to users from CA</strong> </li>
</ul>



<p><a id="conclusions"></a></p>
<h2 id="Finishing-Up">Finishing Up<a class="anchor-link" href="#Finishing-Up">¶</a></h2><blockquote><p>Congratulations!  You have reached the end of the A/B Test Results project!  You should be very proud of all you have accomplished!</p>
<p><strong>Tip</strong>: Once you are satisfied with your work here, check over your report to make sure that it is satisfies all the areas of the rubric (found on the project submission page at the end of the lesson). You should also probably remove all of the "Tips" like this one so that the presentation is as polished as possible.</p>
</blockquote>
<h2 id="Directions-to-Submit">Directions to Submit<a class="anchor-link" href="#Directions-to-Submit">¶</a></h2><blockquote><p>Before you submit your project, you need to create a .html or .pdf version of this notebook in the workspace here. To do that, run the code cell below. If it worked correctly, you should get a return code of 0, and you should see the generated .html file in the workspace directory (click on the orange Jupyter icon in the upper left).</p>
<p>Alternatively, you can download this report as .html via the <strong>File</strong> &gt; <strong>Download as</strong> submenu, and then manually upload it into the workspace directory by clicking on the orange Jupyter icon in the upper left, then using the Upload button.</p>
<p>Once you've done this, you can submit your project by clicking on the "Submit Project" button in the lower right here. This will create and submit a zip file with this .ipynb doc and the .html or .pdf version you created. Congratulations!</p>
</blockquote>


In [ ]:

from subprocess import call
call(['python', '-m', 'nbconvert', 'Analyze_ab_test_results_notebook.ipynb'])

